# Credit Risk Resampling Techniques

In [1]:
# import warnings to ignore system generated warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Initial imports and their dependencies. Note: SKlearn imports are below with the code
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [4]:
# We will use LabelEncoder to transform our target values
# Note: In this exercise, I will import all the necessary libraries at the cell levels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [5]:
# Fitting and encoding the columns with the LabelEncoder, loan_status column - This will transform 
# loan_status column categories 'low_risk' to 1 and 'high_risk' to 0
le.fit(df['loan_status'])
df['loan_status'] = le.transform(df['loan_status'])

In [6]:
# we will use pandas get_dummies function to separate series values (non-nominal) into new columns
df = pd.get_dummies(df, columns=['homeowner'])

In [7]:
# Show all the columns to check if homeowner column spawned into new homeowner columns
df.columns

Index(['loan_size', 'interest_rate', 'borrower_income', 'debt_to_income',
       'num_of_accounts', 'derogatory_marks', 'total_debt', 'loan_status',
       'homeowner_mortgage', 'homeowner_own', 'homeowner_rent'],
      dtype='object')

# Split the Data into Training and Testing

In [8]:
# Create our features
X = df.drop(columns=['loan_status'])

# Create our target
y = df['loan_status']

In [9]:
# use describe function to get summary of our features
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [10]:
# Check the balance of our target values
y.value_counts()

1    75036
0     2500
Name: loan_status, dtype: int64

In [11]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [12]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [13]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [14]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [15]:
# Use Logistic Regression model - I will use different naming conventions for our models to distinguish 
# among our other models below
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression(solver='lbfgs', random_state=42)
model1.fit(X_train_scaled, y_train)

LogisticRegression(random_state=42)

In [16]:
# Make the prediction by using scaled data
y_pred1 = model1.predict(X_test_scaled)

In [17]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred1)

0.9879955386420904

In [18]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred1)

array([[  581,    11],
       [  102, 18690]])

In [19]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred1))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      0.98      0.99      0.91      0.99      0.97       592
          1       1.00      0.99      0.98      1.00      0.99      0.98     18792

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [20]:
# Check the balance of target y for later comparison
y_train.value_counts()

1    56244
0     1908
Name: loan_status, dtype: int64

In [21]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_resampled2, y_resampled2 = ros.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter 
Counter(y_resampled2)

Counter({1: 56244, 0: 56244})

In [22]:
# On my mac, Counter didn't render any counts for our target class , so i will use value_counts
y_resampled2.value_counts()

0    56244
1    56244
Name: loan_status, dtype: int64

In [23]:
# Train the Logistic Regression model using the resampled data
model2 = LogisticRegression(solver='lbfgs', random_state=42)
model2.fit(X_resampled2, y_resampled2)

LogisticRegression(random_state=42)

In [24]:
# Predict and display the confusion matrix
y_pred2 = model2.predict(X_test_scaled)
confusion_matrix(y_test, y_pred2)

array([[  590,     2],
       [  115, 18677]])

In [25]:
# Calculate the balanced accuracy score
balanced_accuracy_score(y_test, y_pred2)

0.9952509981245614

In [26]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred2))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.84      1.00      0.99      0.91      1.00      0.99       592
          1       1.00      0.99      1.00      1.00      1.00      0.99     18792

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



### SMOTE Oversampling

In [27]:
# Check the balance of target y for later comparison
y_train.value_counts()

1    56244
0     1908
Name: loan_status, dtype: int64

In [28]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled3, y_resampled3 = SMOTE(random_state=42, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train
)

# View the count of target classes with value_counts
y_resampled3.value_counts()

0    56244
1    56244
Name: loan_status, dtype: int64

In [29]:
# Train the Logistic Regression model using the resampled data
model3 = LogisticRegression(solver='lbfgs', random_state=42)
model3.fit(X_resampled3, y_resampled3)

LogisticRegression(random_state=42)

In [30]:
# Predict and calculate the balanced accuracy score
y_pred3 = model3.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred3)

0.9952509981245614

In [31]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred3)

array([[  590,     2],
       [  115, 18677]])

In [32]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred3))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.84      1.00      0.99      0.91      1.00      0.99       592
          1       1.00      0.99      1.00      1.00      1.00      0.99     18792

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [33]:
# Check the balance of target y for later comparison
y_train.value_counts()

1    56244
0     1908
Name: loan_status, dtype: int64

In [34]:
# Resample the data using the ClusterCentroids resampler - This will take some time to run
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=42)

# fit train data to run ClusterCentroids (cc)
X_resampled4, y_resampled4 = cc.fit_resample(X_train_scaled, y_train)

# View the count of target classes with value_counts
y_resampled4.value_counts()

0    1908
1    1908
Name: loan_status, dtype: int64

In [35]:
# Train the Logistic Regression model using the resampled data
model4 = LogisticRegression(solver='lbfgs', random_state=42, max_iter=2000)
model4.fit(X_resampled4, y_resampled4)

LogisticRegression(max_iter=2000, random_state=42)

In [36]:
# predict and display the confusion matrix
y_pred4 = model4.predict(X_test_scaled)
confusion_matrix(y_test, y_pred4)

array([[  588,     4],
       [  103, 18689]])

In [37]:
# Calculate the balanced accuracy score
balanced_accuracy_score(y_test, y_pred4)

0.9938810937374156

In [38]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred4))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.85      0.99      0.99      0.92      0.99      0.99       592
          1       1.00      0.99      0.99      1.00      0.99      0.99     18792

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [39]:
# Check the balance of target y for later comparison
y_train.value_counts()

1    56244
0     1908
Name: loan_status, dtype: int64

In [40]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=42)
X_resampled5, y_resampled5 = smote_enn.fit_resample(X_train_scaled, y_train)


# View the count of target classes with value_counts
y_resampled5.value_counts()

1    55867
0    55603
Name: loan_status, dtype: int64

In [41]:
# Train the Logistic Regression model using the resampled data
model5 = LogisticRegression(solver='lbfgs', random_state=42)
model5.fit(X_resampled5, y_resampled5)

LogisticRegression(random_state=42)

In [42]:
# predict and display the confusion matrix
y_pred5 = model5.predict(X_test_scaled)
confusion_matrix(y_test, y_pred5)

array([[  590,     2],
       [  116, 18676]])

In [43]:
# Calculate the balanced accuracy score
balanced_accuracy_score(y_test, y_pred5)

0.9952243910577243

In [44]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred5))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.84      1.00      0.99      0.91      1.00      0.99       592
          1       1.00      0.99      1.00      1.00      1.00      0.99     18792

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?
    
     ANSWER below in takeaways

2. Which model had the best recall score?

     ANSWER below in takeaways

3. Which model had the best geometric mean score?

     ANSWER below in takeaways


# Takeaways

### Answer 1

Simple logistic Regression model (without using any sampling models) had a balanced accuracy score of 0.9879955386420904. Random OverSampling  and SMOTE Oversampling models had the better balanced accuracy score of 0.9952509981245614.

### Answer 2

All models had the same recall score of 0.99.

### Answer 3

Oversampling, SMOTE Oversampling, and SMOTEENN Combination sampling all had geometric mean score of 1.00 which was better than the simple logistic regression model geometric mean score of 0.99.